In [28]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import json

In [29]:
from matplotlib import rc
rc('font', family='sans-serif')
rc('font', size=13.0)
rc('text', usetex=False)
rc('figure', figsize=(11.69,8.27))

from matplotlib.font_manager import FontProperties

panel_label_font = FontProperties().copy()
panel_label_font.set_weight("bold")
panel_label_font.set_size(14.0)
panel_label_font.set_family("sans-serif")

# Seaborn styles
sns.set_style("whitegrid")

In [30]:
df = pd.read_csv("paperbuzz.csv", dtype={'status':str})
df.set_index("id", inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = pd.to_datetime(df['date'], errors = 'coerce')

df = df[~df.date.isnull()]

In [38]:
df.status.value_counts()

200            6171
ReadTimeout      27
500               2
Name: status, dtype: int64

In [41]:
results = df.copy()

for ix, row in df.iterrows():
    if row['status'] == "200":
        j = json.loads(row['response'])
        if 'altmetrics_sources' in j:
            for s in j['altmetrics_sources']:
                name = s['source_id']
                if name in results.columns:
                    results.loc[ix, name] = float(s['events_count'])
                else:
                    results[name] = np.nan
                    results.loc[ix, name] = float(s['events_count'])

In [45]:
results.describe()

,twitter,wikipedia,datacite,wordpressdotcom
count,201.000000,5.000000,38.000000,1.0
mean,4.532338,4.600000,3.342105,1.0
std,7.431702,4.774935,5.463769,NaN
min,1.000000,2.000000,1.000000,1.0
25%,1.000000,2.000000,2.000000,1.0
50%,2.000000,2.000000,2.000000,1.0
75%,5.000000,4.000000,2.000000,1.0
max,59.000000,13.000000,31.000000,1.0
